In [1]:
import pandas as pd
import numpy as np
from numpy import nan 
import random

In [2]:
order_df = pd.read_csv('order.table.xls',header=0,sep='\t')
dna_df = pd.read_csv('DNA.table.xls',header=0,sep='\t')
delivery_df = pd.read_csv('delivery.table.xls',header=0,sep='\t')
 

In [3]:
# order_df
# dna_df
# delivery_df

In [4]:
order_df['receive_date'] = delivery_df['receive_date']

In [5]:
# order_df['receive_date'] 

In [6]:
order_df['receive_date'] = order_df['receive_date'].str.strip(' 00:00:00').str.replace('-','.')

In [7]:
# 部分样品编码含有反斜杠，这些反斜杠在数据保存的路径被‘-’代替了 
order_df['sample_code_format'] =  order_df['sample_code'].str.replace('/','-').str.replace(' ','-').str.replace(u'加急','jiaji')
delivery_df['sample_code'] = delivery_df['sample_code'].str.replace('/','-').str.replace(' ','-')
dna_df['sample_code'] = dna_df['sample_code'].str.replace('/','-').str.replace(' ','-')


In [8]:
order_df.iloc[1958,:]

sample_code                  ZBAD2021-加急1
detection_items                飞鸟残骸鉴定（加急）
department                       北京大兴国际机场
order_date            2021-04-28 00:00:00
order_num                             NaN
report_date                           NaN
conclusion                            NaN
receive_date                    2021.04.3
sample_code_format        ZBAD2021-jiaji1
Name: 1958, dtype: object

In [9]:
# 找到那些不含有日期的字符，使他为nan：
order_df[order_df['receive_date'] == '.'] = nan
# 填充空缺的日期：
order_df['receive_date'].fillna(method='ffill',axis=0,inplace=True)



In [10]:
# order_df[order_df['receive_date'].str.contains('\b\.\b')]

# 由于日期格式不统一，所以 将字符串都转为日期类型：
order_df['infer_receive_date'] = order_df['receive_date'].apply(pd.to_datetime,infer_datetime_format=True)



In [11]:
# order_df['receive_date'].apply(pd.to_datetime,infer_datetime_format=True)
# 再将日期转回字符串：
order_df['receive_date'] = order_df['infer_receive_date'].apply(lambda x : x.strftime('%Y.%m.%d'))

order_df['order_date'].fillna(order_df['receive_date'],inplace=True)#
order_df['order_date'] = order_df['order_date'].str.strip(' 00:00:00').str.replace('-','.')
order_df['report_date'].fillna(order_df['receive_date'],inplace=True)
# order_df['report_date'] =order_df['report_date'].apply(lambda x : x.strftime('%Y.%m.%d')).str.strip(' 00:00:00').str.replace('-','.')
order_df['create_time'] = order_df['report_date']
order_df['sample_date'] = order_df['report_date']

order_df.columns

Index(['sample_code', 'detection_items', 'department', 'order_date',
       'order_num', 'report_date', 'conclusion', 'receive_date',
       'sample_code_format', 'infer_receive_date', 'create_time',
       'sample_date'],
      dtype='object')

In [12]:
order_df[pd.isna(order_df['order_date'])]

,sample_code,detection_items,department,order_date,order_num,report_date,conclusion,receive_date,sample_code_format,infer_receive_date,create_time,sample_date


In [13]:
order_df

,sample_code,detection_items,department,order_date,order_num,report_date,conclusion,receive_date,sample_code_format,infer_receive_date,create_time,sample_date
0,ZSHC/2017/81,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,NaN,2018.04.16,NaN,2018.04.16,ZSHC-2017-81,2018-04-16,2018.04.16,2018.04.16
1,ZSHC/2017/80,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,NaN,2018.04.16,NaN,2018.04.16,ZSHC-2017-80,2018-04-16,2018.04.16,2018.04.16
2,ZSHC/2017/82,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,NaN,2018.04.16,NaN,2018.04.16,ZSHC-2017-82,2018-04-16,2018.04.16,2018.04.16
3,ZSHC/2017/83,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,NaN,2018.04.16,NaN,2018.04.16,ZSHC-2017-83,2018-04-16,2018.04.16,2018.04.16
4,ZSHC/2018/01,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,NaN,2018.04.16,NaN,2018.04.16,ZSHC-2018-01,2018-04-16,2018.04.16,2018.04.16
...,...,...,...,...,...,...,...,...,...,...,...,...
2022,20210429-0-014,飞鸟残骸鉴定（常规）,南宁吴圩国际机场,2021.05.1,NaN,2021.05.12,NaN,2021.05.12,20210429-0-014,2021-05-12,2021.05.12,2021.05.12
2023,20210429-0-015,飞鸟残骸鉴定（常规）,南宁吴圩国际机场,2021.05.1,NaN,2021.05.12,NaN,2021.05.12,20210429-0-015,2021-05-12,2021.05.12,2021.05.12
2024,20210429-0-016,飞鸟残骸鉴定（常规）,南宁吴圩国际机场,2021.05.1,NaN,2021.05.12,NaN,2021.05.12,20210429-0-016,2021-05-12,2021.05.12,2021.05.12
2025,20210429-0-017,飞鸟残骸鉴定（常规）,南宁吴圩国际机场,2021.05.1,NaN,2021.05.12,NaN,2021.05.12,20210429-0-017,2021-05-12,2021.05.12,2021.05.12


In [14]:
length_df = len(order_df['receive_date'])
order_df['random_num'] = [str(random.randint(10000, 99999)) for i in range(length_df) ]

In [15]:
# order_df.random_num
order_df['order_num'] = "SEQ"+order_df['receive_date'].str.replace('\.','')+ order_df['random_num'] 

order_df['order_num'] 

/ifs1/Reotech/project/www/bird_strike/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


0       SEQ2018041676063
1       SEQ2018041686582
2       SEQ2018041672814
3       SEQ2018041654900
4       SEQ2018041636426
              ...       
2022    SEQ2021051283046
2023    SEQ2021051221950
2024    SEQ2021051268581
2025    SEQ2021051234176
2026    SEQ2021051258503
Name: order_num, Length: 2027, dtype: object

In [16]:
# 由于有部分测序路径的文件名是以航班号而非样本名来命名，所以有必要吧航班号放进来：
order_df['event_flight']  = delivery_df['event_flight']

In [17]:
# 测试填空值：
# df = pd.DataFrame(np.random.randn(6,3))
# df.iloc[1:4,1] = nan
# df.iloc[2:5,2] = nan
# df.fillna(method='ffill',axis=0)

In [18]:
order_df['id'] = order_df.index +1 
# del order_df['random_num']
# del order_df['receive_date']
order_df.to_csv('order_df.tsv',encoding='utf_8_sig',index=False)
print('---1')

---1


In [19]:
#有些样品是没有样品名的，先挑出来，以后在处理这些样品。
order_df_null_samplecode = order_df[order_df['sample_code'].isnull()]
order_df_null_samplecode.to_csv('order_df_null_samplecode.tsv',encoding='utf_8_sig',index=False)

In [20]:
# order_df.loc[[8,9,10],'sample_code']

# 有两行的sample_code 在总表里不为空，但在这里为空了，估计是复制的时候操作失误？
order_df.iloc[np.r_[1152:1157],:]

,sample_code,detection_items,department,order_date,order_num,report_date,conclusion,receive_date,sample_code_format,infer_receive_date,create_time,sample_date,random_num,event_flight,id
1152,ZGGG-2020-63,飞鸟残骸鉴定（常规）,广州白云国际机场飞行区管理部生态管理部,2020.07.21,SEQ2020072347183,2020.07.23,NaN,2020.07.23,ZGGG-2020-63,2020-07-23,2020.07.23,2020.07.23,47183,NaN,1153
1153,NaN,NaN,NaN,2020.07.23,SEQ2020072389036,2020.07.23,NaN,2020.07.23,NaN,2020-07-23,2020.07.23,2020.07.23,89036,NaN,1154
1154,ZGGG-2020-74,飞鸟残骸鉴定（常规）,广州白云国际机场飞行区管理部生态管理部,2020.07.21,SEQ2020072333616,2020.07.23,NaN,2020.07.23,ZGGG-2020-74,2020-07-23,2020.07.23,2020.07.23,33616,NaN,1155
1155,NaN,NaN,NaN,2020.07.23,SEQ2020072386291,2020.07.23,NaN,2020.07.23,NaN,2020-07-23,2020.07.23,2020.07.23,86291,NaN,1156
1156,ZGGG-2020-77,飞鸟残骸鉴定（常规）,广州白云国际机场飞行区管理部生态管理部,2020.07.21,SEQ2020072354807,2020.07.23,NaN,2020.07.23,ZGGG-2020-77,2020-07-23,2020.07.23,2020.07.23,54807,NaN,1157


In [21]:
order_df.iloc[1564,:]

sample_code                  ZBAD2020-282
detection_items                飞鸟残骸鉴定（常规）
department                       北京大兴国际机场
order_date                     2020.09.24
order_num                SEQ2020092638058
report_date                    2020.09.26
conclusion                            NaN
receive_date                   2020.09.26
sample_code_format           ZBAD2020-282
infer_receive_date    2020-09-26 00:00:00
create_time                    2020.09.26
sample_date                    2020.09.26
random_num                          38058
event_flight                       KN5912
id                                   1565
Name: 1564, dtype: object

In [22]:
dna_df.to_csv('dna_df.tsv',encoding='utf_8_sig',index=False)

In [23]:
delivery_df.to_csv('delivery_df.tsv',encoding='utf_8_sig',index=False)

In [24]:
path_df = pd.read_csv('airport.csv',header=0,sep=',')

In [25]:
p = path_df.iloc[1402].sample_code
# print(p.str.split('/'))
print(p)

ZBAD2020-300


In [26]:
result_df = pd.merge(order_df,path_df,left_on=['sample_code_format'],right_on=['sample_code'],how='outer')
result_df_event_flight = pd.merge(order_df,path_df,left_on=['event_flight'],right_on=['sample_code'],how='inner') # 

In [27]:
result_df.count()

sample_code_x         3234
detection_items       3249
department            3249
order_date            3251
order_num             3251
report_date           3251
conclusion               0
receive_date          3251
sample_code_format    3234
infer_receive_date    3251
create_time           3251
sample_date           3251
random_num_x          3251
event_flight          2332
id                    3251
Unnamed: 0            2778
date                  2778
code                  2778
gene_name             2778
file_name             2778
path                  2778
sample_code_y         2778
random_num_y          2778
data_num              2778
dtype: int64

In [28]:
result_df['sample_code'] = result_df['sample_code_format']
# 删除含有空的sample_code的行：
result_df = result_df.dropna(subset=['sample_code'])
result_df.count()
# result_df.head(20)

sample_code_x         3234
detection_items       3234
department            3234
order_date            3234
order_num             3234
report_date           3234
conclusion               0
receive_date          3234
sample_code_format    3234
infer_receive_date    3234
create_time           3234
sample_date           3234
random_num_x          3234
event_flight          2318
id                    3234
Unnamed: 0            2634
date                  2634
code                  2634
gene_name             2634
file_name             2634
path                  2634
sample_code_y         2634
random_num_y          2634
data_num              2634
sample_code           3234
dtype: int64

In [29]:
result_df.to_csv('result_df.tsv',encoding='utf_8_sig',index=False)
result_df_event_flight.to_csv('result_df_event_flight.tsv',encoding='utf_8_sig',index=False)

In [30]:
# result_df.loc[:,['department','order_date']]
# result_df.loc[:,['order_date']].fillna(result_df.loc[:,['receive_date']],inplace=True)
# result_df.loc[:,['order_date']].fillna(method='pad',inplace=True)
result_df.loc[:,['order_date']].fillna(method='pad')

,order_date
0,2018.4.9
1,2018.4.9
2,2018.4.9
3,2018.4.9
4,2018.4.9
...,...
3246,2021.05.1
3247,2021.05.1
3248,2021.05.1
3249,2021.05.1


In [31]:
result_df

,sample_code_x,detection_items,department,order_date,order_num,report_date,conclusion,receive_date,sample_code_format,infer_receive_date,...,Unnamed: 0,date,code,gene_name,file_name,path,sample_code_y,random_num_y,data_num,sample_code
0,ZSHC/2017/81,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,SEQ2018041676063,2018.04.16,NaN,2018.04.16,ZSHC-2017-81,2018-04-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZSHC-2017-81
1,ZSHC/2017/80,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,SEQ2018041686582,2018.04.16,NaN,2018.04.16,ZSHC-2017-80,2018-04-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZSHC-2017-80
2,ZSHC/2017/82,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,SEQ2018041672814,2018.04.16,NaN,2018.04.16,ZSHC-2017-82,2018-04-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZSHC-2017-82
3,ZSHC/2017/83,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,SEQ2018041654900,2018.04.16,NaN,2018.04.16,ZSHC-2017-83,2018-04-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZSHC-2017-83
4,ZSHC/2018/01,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,SEQ2018041636426,2018.04.16,NaN,2018.04.16,ZSHC-2018-01,2018-04-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZSHC-2018-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3246,20210429-0-014,飞鸟残骸鉴定（常规）,南宁吴圩国际机场,2021.05.1,SEQ2021051283046,2021.05.12,NaN,2021.05.12,20210429-0-014,2021-05-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210429-0-014
3247,20210429-0-015,飞鸟残骸鉴定（常规）,南宁吴圩国际机场,2021.05.1,SEQ2021051221950,2021.05.12,NaN,2021.05.12,20210429-0-015,2021-05-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210429-0-015
3248,20210429-0-016,飞鸟残骸鉴定（常规）,南宁吴圩国际机场,2021.05.1,SEQ2021051268581,2021.05.12,NaN,2021.05.12,20210429-0-016,2021-05-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210429-0-016
3249,20210429-0-017,飞鸟残骸鉴定（常规）,南宁吴圩国际机场,2021.05.1,SEQ2021051234176,2021.05.12,NaN,2021.05.12,20210429-0-017,2021-05-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210429-0-017


In [32]:
# result_df['sample_code'] = result_df['sample_code_format']

In [33]:
result_df['order_date'].fillna(result_df['infer_receive_date'],inplace=True)
result_df['report_date'].fillna(result_df['infer_receive_date'],inplace=True)


In [34]:
result_df[pd.isna(result_df['order_date'])]
# test_fill = result_df[pd.isna(result_df['sample_code_format'])]
result_df.to_csv('result_df_not_null_date.csv',encoding='utf_8_sig',index=False)

In [35]:
s = 'ZBAD2021-加急1'
s.replace('加急','jiaji')

'ZBAD2021-jiaji1'

In [36]:
result_df.head(10)

,sample_code_x,detection_items,department,order_date,order_num,report_date,conclusion,receive_date,sample_code_format,infer_receive_date,...,Unnamed: 0,date,code,gene_name,file_name,path,sample_code_y,random_num_y,data_num,sample_code
0,ZSHC/2017/81,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,SEQ2018041676063,2018.04.16,NaN,2018.04.16,ZSHC-2017-81,2018-04-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZSHC-2017-81
1,ZSHC/2017/80,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,SEQ2018041686582,2018.04.16,NaN,2018.04.16,ZSHC-2017-80,2018-04-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZSHC-2017-80
2,ZSHC/2017/82,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,SEQ2018041672814,2018.04.16,NaN,2018.04.16,ZSHC-2017-82,2018-04-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZSHC-2017-82
3,ZSHC/2017/83,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,SEQ2018041654900,2018.04.16,NaN,2018.04.16,ZSHC-2017-83,2018-04-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZSHC-2017-83
4,ZSHC/2018/01,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,SEQ2018041636426,2018.04.16,NaN,2018.04.16,ZSHC-2018-01,2018-04-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZSHC-2018-01
5,ZSHC/2018/02,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,SEQ2018041654953,2018.04.16,NaN,2018.04.16,ZSHC-2018-02,2018-04-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZSHC-2018-02
6,ZSHC/2018/03,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,SEQ2018041665802,2018.04.16,NaN,2018.04.16,ZSHC-2018-03,2018-04-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZSHC-2018-03
7,ZSHC/2018/04,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,SEQ2018041672050,2018.04.16,NaN,2018.04.16,ZSHC-2018-04,2018-04-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZSHC-2018-04
8,ZGGG-2018-07,飞鸟残骸鉴定（常规）,广州白云国际机场,2018.05.09,SEQ2018050932167,2018.05.09,NaN,2018.05.09,ZGGG-2018-07,2018-05-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZGGG-2018-07
9,ZGGG-2018-07,飞鸟残骸鉴定（常规）,广州白云国际机场,2018.05.09,SEQ2018050993212,2018.05.09,NaN,2018.05.09,ZGGG-2018-07,2018-05-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZGGG-2018-07


In [37]:
# order_df.iloc[[1849,1850],:]

In [38]:
print('--')

--


In [39]:
result_df.path

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
3246    NaN
3247    NaN
3248    NaN
3249    NaN
3250    NaN
Name: path, Length: 3234, dtype: object

In [40]:
from app import create_app
app = create_app()
app_context =  app.app_context()
app_context.push()



from app.models1 import *
from app.models import *
import random
from datetime import datetime

In [41]:
result_df['id'] = result_df['id'].apply(lambda x : int(x) if not pd.isnull(x) else 0)
# 导入数据库

In [42]:

from app import create_app
app = create_app()
app_context =  app.app_context()
app_context.push()

from app.models1 import *
from app.models import *
import random
from datetime import datetime


In [43]:
# result_df['create_time'] = pd.to_datetime(result_df['create_time'],infer_datetime_format=True )
# result_df['sample_date'] = pd.to_datetime(result_df['sample_date'],infer_datetime_format=True )
# result_df['receive_date'] = pd.to_datetime(result_df['receive_date'],infer_datetime_format=True )
# result_df['report_date'] = pd.to_datetime(result_df['report_date'],infer_datetime_format=True )
# print(type(result_df['order_date']))
# result_df['id'] = result_df['id'].apply(lambda x :  int(x)  if x else None  )
# result_df['id'] = result_df['id'].astype('int64')
# print(result_df.id )

print(result_df['id'])

0          1
1          2
2          3
3          4
4          5
        ... 
3246    2023
3247    2024
3248    2025
3249    2026
3250    2027
Name: id, Length: 3234, dtype: int64


In [44]:
old_id = None
i = 0
# lst = []
for ix,val in result_df.iterrows():
    data = val.to_dict()
    id =  data.get('id')
    if not id :
        continue   
    else:
#         print(val.get('id'))        
        
        o = OrderSeq.query.filter_by(id=id).first()
        if not o:
            o = OrderSeq()
            o.from_dict(data)
            db.session.add(o)
#由于测序数据与订单不是直接的对应关系，因此下列的代码不合理        
#     if val.get('path') and not pd.isnull(val.get('path')):
#         seqData = SeqData()
#         seqData.from_dict(data)
 
#         with open (val.get('path')) as fh:
#             raw_data = fh.read()
#             seqData.raw_data = raw_data 
            
# #         o.seqData.append(seqData)
        
#         db.session.add(seqData)
    
#     old_id = data.get('id')
    
    i+= 1
    if i%100 == 0:
#         print(lst)
        db.session.commit()
        
db.session.commit()
print('finish ')
# app_context.pop()        
        

    
    

finish 


In [45]:
path_df


,Unnamed: 0,date,code,gene_name,file_name,path,sample_code,random_num,data_num
0,0,20200427,ZSHC,cytb,ZSHC-2020-07_L14995-H16065_C2291.L14995.PW0050...,bird_strike/20200427&20200506潮汕机场2个萧山机场7个样品结果/...,ZSHC-2020-07,40749,ZSHCcytb2020042740749
1,1,20200427,ZSHC,cytb,ZSHC-2020-04_L14995-H16065_C2291.L14995.PW0050...,bird_strike/20200427&20200506潮汕机场2个萧山机场7个样品结果/...,ZSHC-2020-04,26039,ZSHCcytb2020042726039
2,2,20200427,ZSHC,cytb,ZSHC-2020-02_L14995-H16065_C2291.L14995.PW0050...,bird_strike/20200427&20200506潮汕机场2个萧山机场7个样品结果/...,ZSHC-2020-02,37879,ZSHCcytb2020042737879
3,3,20200427,JYCS,cytb,JYCS2020-03_L14995-H16065_C2291.L14995.PW00509...,bird_strike/20200427&20200506潮汕机场2个萧山机场7个样品结果/...,JYCS2020-03,76260,JYCScytb2020042776260
4,4,20200427,ZSHC,cytb,ZSHC-2020-03_L14995-H16065_C2291.L14995.PW0050...,bird_strike/20200427&20200506潮汕机场2个萧山机场7个样品结果/...,ZSHC-2020-03,84663,ZSHCcytb2020042784663
...,...,...,...,...,...,...,...,...,...
2581,2581,20200825,KN,cytb,KN2928_L14995-H16065_C2777.L14995.PW009022109.seq,bird_strike/20200825&0826萧山机场4个&北京大兴机场44个样品结果/...,KN2928,12815,KNcytb2020082512815
2582,2582,20200825,MF,cytb,MF8169_L14995-H16065_C2777.L14995.PW009022080.seq,bird_strike/20200825&0826萧山机场4个&北京大兴机场44个样品结果/...,MF8169,21483,MFcytb2020082521483
2583,2583,20200825,CZ,cytb,CZ6715_L14995-H16065_C2777.L14995.PW009022092.seq,bird_strike/20200825&0826萧山机场4个&北京大兴机场44个样品结果/...,CZ6715,92233,CZcytb2020082592233
2584,2584,20200413,JYCS,cytb,JYCS2020-01_L14995-H16065_C2243.L14995.PW00417...,bird_strike/20200413潮汕机场1个样品结果/测序结果/cytb/JYCS2...,JYCS2020-01,56870,JYCScytb2020041356870


In [46]:
# 导入Delivery表
delivery_df['id'] = delivery_df.index + 1 

dna_df[id] = dna_df.index + 1 

for ix,val in delivery_df.iterrows():
    data = val.to_dict()
    id =  data.get('id')
    if not id :
        
        continue   
    else:
#         print(val.get('id'))        
        
        o = OrderSeq.query.filter_by(id=id).first()
        if not o:
            print('can not find order: '+str(id))
            continue
         
#     if val.get('path') and not pd.isnull(val.get('path')):
        delivery = Delivery()
        delivery.from_dict(data)
             
        o.delivery.append(delivery)
        db.session.add(delivery)
    
#     old_id = data.get('id')
    
    i+= 1
    if i%100 == 0:
#         print(lst)
        db.session.commit()
        
db.session.commit()
print('finish ')
# app_context.pop()   

can not find order: 10
can not find order: 13
can not find order: 105
can not find order: 106
can not find order: 203
can not find order: 204
can not find order: 205
can not find order: 206
can not find order: 207
can not find order: 208
can not find order: 209
can not find order: 210
can not find order: 211
can not find order: 212
can not find order: 293
can not find order: 1154
can not find order: 1156
finish 


In [47]:
# -----------------------
# todo


#测序结果是与delivery直接关联的：
old_id = None
i = 0
# lst = []
for ix,val in path_df.iterrows():
    data = val.to_dict()
    if val.get('path') and not pd.isnull(val.get('path')):
        seqData = SeqData()
        seqData.from_dict(data)

        with open (data.get('path')) as fh:
            raw_data = fh.read()
            seqData.raw_data = raw_data 

    #         o.seqData.append(seqData)
        
        d = Delivery.query.filter_by(sample_code=data.get('sample_code')).first()
        if not d:
            print('can not find Delivery: '+data.get('sample_code'))
            continue
        else:
            seqData.delivery = d 
        db.session.add(seqData)


        i+= 1
        if i%100 == 0:
            db.session.commit()
        
db.session.commit()

can not find Delivery: ZBAD2021-jiaji1
can not find Delivery: ZBAD2021-jiaji1
can not find Delivery: ZBAD2021-jiaji1
can not find Delivery: ZBAD2021-jiaji1
can not find Delivery: ZBAD2021-jiaji1
can not find Delivery: ZBAD2021-jiaji1
can not find Delivery: ZBAD2021-jiaji1
can not find Delivery: ZBAD2021-jiaji1
can not find Delivery: ZBAD2021-jiaji1
can not find Delivery: ZBAD2021-jiaji1
can not find Delivery: ZGGG-2020-194
can not find Delivery: ZGGG-2020-185
can not find Delivery: ZGGG-2020-114
can not find Delivery: ZGGG-2020-155
can not find Delivery: ZGGG-2020-149
can not find Delivery: ZGGG-2020-108
can not find Delivery: ZGGG-2020-150
can not find Delivery: ZGGG-2020-135
can not find Delivery: ZGGG-2020-136
can not find Delivery: ZGGG-2020-187
can not find Delivery: ZGGG-2020-161
can not find Delivery: ZGGG-2020-170
can not find Delivery: ZGGG-2020-130
can not find Delivery: ZGGG-2020-105
can not find Delivery: ZGGG-2020-134
can not find Delivery: ZGGG-2020-159
can not find Deliv

In [48]:
d= Delivery.query.filter_by(sample_code='ZGGG-2020-194').first()
print(d)

None


In [49]:
# OrderSeq.__table__.columns.keys()
# db.session.rollback()

In [50]:
# 导入dna_df 和 delivery_df 

In [51]:
# 导入dna_df表
dna_df['id'] = dna_df.index + 1 
# print(dna_df)


for ix,val in dna_df.iterrows():
    data = val.to_dict()
#     print(data)
    id =  data.get('id')
    if not id :
        continue   
    else:
#         print(val.get('id'))        
        
        d = Delivery.query.filter_by(id=id).first()
        if not d:
            print('can not find Delivery: '+str(id))
            continue
         
#     if val.get('path') and not pd.isnull(val.get('path')):
        dnaInfo = DNAInfo()
        dnaInfo.from_dict(data)
             
        d.DNA_Info.append(dnaInfo)
        db.session.add(dnaInfo)
    
#     old_id = data.get('id')
    
    i+= 1
    if i%100 == 0:
#         print(lst)
        db.session.commit()
        
db.session.commit()
print('finish ')
# app_context.pop()        

# db.session.rollback()

can not find Delivery: 10
can not find Delivery: 13
can not find Delivery: 105
can not find Delivery: 106
can not find Delivery: 203
can not find Delivery: 204
can not find Delivery: 205
can not find Delivery: 206
can not find Delivery: 207
can not find Delivery: 208
can not find Delivery: 209
can not find Delivery: 210
can not find Delivery: 211
can not find Delivery: 212
can not find Delivery: 293
can not find Delivery: 1154
can not find Delivery: 1156
finish 


In [52]:


# 订单关联到各个渠道
# result_df.head()
# result_df.describe()
result_df.columns

Index(['sample_code_x', 'detection_items', 'department', 'order_date',
       'order_num', 'report_date', 'conclusion', 'receive_date',
       'sample_code_format', 'infer_receive_date', 'create_time',
       'sample_date', 'random_num_x', 'event_flight', 'id', 'Unnamed: 0',
       'date', 'code', 'gene_name', 'file_name', 'path', 'sample_code_y',
       'random_num_y', 'data_num', 'sample_code'],
      dtype='object')

由于订单里的渠道名不统一，且同一个渠道还有不同的名，在此统一一下：
```
珠港机场管理有限公司	珠港机场管理有限公司
重庆机场集团公司飞行区管理部	重庆机场集团公司飞行区管理部
郑州新郑国际机场场道管理部鸟击防范室	郑州新郑国际机场场道管理部鸟击防范室
浙江省义乌机场管理有限公司	浙江省义乌机场管理有限公司
长沙黄花国际机场分公司飞行区管理部	长沙黄花国际机场分公司飞行区管理部
徐州机场	徐州机场
武汉机场	武汉机场
天津滨海国际机场	天津滨海国际机场
深圳市机场股份有限公司	深圳机场
深圳机场股份有限公司	深圳机场
深圳机场	深圳机场
上海浦东国际机场	上海浦东国际机场
山东临沂机场	山东临沂机场
南通兴东国际机场	南通兴东国际机场
南宁吴圩国际机场	南宁吴圩国际机场
南宁机场修缮动力部	南宁吴圩国际机场
揭阳潮汕机场	揭阳潮汕机场
杭州萧山国际机场有限公司	杭州萧山国际机场
杭州萧山国际机场股份公司飞行区管理中心	杭州萧山国际机场
杭州萧山国际机场	杭州萧山国际机场
广州白云国际机场股份有限公司	广州白云国际机场
广州白云国际机场飞行区管理部生态管理部	广州白云国际机场
广州白云国际机场	广州白云国际机场
大连周水子国际机场	大连周水子国际机场
大连国际机场股份公司飞行区管理部	大连周水子国际机场
常州国际机场有限公司	常州国际机场有限公司
北京大兴国际机场	北京大兴国际机场

```

In [53]:
diff_lst = ['珠港机场管理有限公司','重庆机场集团公司飞行区管理部','郑州新郑国际机场场道管理部鸟击防范室','浙江省义乌机场管理有限公司','长沙黄花国际机场分公司飞行区管理部',
            '徐州机场','武汉机场','天津滨海国际机场','深圳市机场股份有限公司','深圳机场股份有限公司','深圳机场','上海浦东国际机场','山东临沂机场','南通兴东国际机场',
            '南宁吴圩国际机场','南宁机场修缮动力部','揭阳潮汕机场','杭州萧山国际机场有限公司','杭州萧山国际机场股份公司飞行区管理中心','杭州萧山国际机场','广州白云国际机场股份有限公司',
            '广州白云国际机场飞行区管理部生态管理部','广州白云国际机场','大连周水子国际机场','大连国际机场股份公司飞行区管理部','常州国际机场有限公司','北京大兴国际机场']

same_lst = ['珠港机场管理有限公司','重庆机场集团公司飞行区管理部','郑州新郑国际机场场道管理部鸟击防范室','浙江省义乌机场管理有限公司','长沙黄花国际机场分公司飞行区管理部',
'徐州机场','武汉机场','天津滨海国际机场','深圳机场','深圳机场','深圳机场','上海浦东国际机场','山东临沂机场','南通兴东国际机场','南宁吴圩国际机场','南宁吴圩国际机场','揭阳潮汕机场',
'杭州萧山国际机场','杭州萧山国际机场','杭州萧山国际机场','广州白云国际机场','广州白云国际机场','广州白云国际机场','大连周水子国际机场','大连周水子国际机场','常州国际机场有限公司','北京大兴国际机场']

airport_dict = dict(zip(diff_lst,same_lst))

print(airport_dict)

{'珠港机场管理有限公司': '珠港机场管理有限公司', '重庆机场集团公司飞行区管理部': '重庆机场集团公司飞行区管理部', '郑州新郑国际机场场道管理部鸟击防范室': '郑州新郑国际机场场道管理部鸟击防范室', '浙江省义乌机场管理有限公司': '浙江省义乌机场管理有限公司', '长沙黄花国际机场分公司飞行区管理部': '长沙黄花国际机场分公司飞行区管理部', '徐州机场': '徐州机场', '武汉机场': '武汉机场', '天津滨海国际机场': '天津滨海国际机场', '深圳市机场股份有限公司': '深圳机场', '深圳机场股份有限公司': '深圳机场', '深圳机场': '深圳机场', '上海浦东国际机场': '上海浦东国际机场', '山东临沂机场': '山东临沂机场', '南通兴东国际机场': '南通兴东国际机场', '南宁吴圩国际机场': '南宁吴圩国际机场', '南宁机场修缮动力部': '南宁吴圩国际机场', '揭阳潮汕机场': '揭阳潮汕机场', '杭州萧山国际机场有限公司': '杭州萧山国际机场', '杭州萧山国际机场股份公司飞行区管理中心': '杭州萧山国际机场', '杭州萧山国际机场': '杭州萧山国际机场', '广州白云国际机场股份有限公司': '广州白云国际机场', '广州白云国际机场飞行区管理部生态管理部': '广州白云国际机场', '广州白云国际机场': '广州白云国际机场', '大连周水子国际机场': '大连周水子国际机场', '大连国际机场股份公司飞行区管理部': '大连周水子国际机场', '常州国际机场有限公司': '常州国际机场有限公司', '北京大兴国际机场': '北京大兴国际机场'}


In [54]:
# 加入各个机场（渠道）信息
for i in range(len(same_lst)):
    c = Channel.query.filter_by(name=same_lst[i]).first()
    if not c :
        c = Channel('',same_lst[i])
        db.session.add(c)
db.session.commit()

In [55]:
# 创建用户信息：
for i in range(len(same_lst)):
    u = User.query.filter_by(username=same_lst[i]).first()
    if not u :
        u = User()
        u.username = same_lst[i]
        u.set_password('123456')
        
        c = Channel.query.filter_by(name=same_lst[i]).first()
        u.channel =c 
        db.session.add(u)
db.session.commit()

In [56]:

# 将订单与渠道管理

order_lst = OrderSeq.query.all()
i = 0
for o in order_lst:
#     print(o.department)
#     print(o)
#     i +=1
#     if i ==5:
#         break
        
    u = User.query.filter_by(username = airport_dict.get( o.department)).first()
    if u:
        o.owner = u
        
db.session.commit()
print('finish')

finish


In [57]:
o = OrderSeq.query.first()
print(o.owner)
# users = User.query.all()
# for u in users:
#     print(u.orders_seq.all())
print('--')

<User 杭州萧山国际机场>
--


In [58]:
o

<OrderSeq 1>